# Debug MNIST

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from tensorflow.examples.tutorials.mnist import input_data
import pdb

np.random.seed(0)

In [ ]:
import plotann
import importlib
importlib.reload(plotann)

# Load Data

In [ ]:
mnist = input_data.read_data_sets('MNIST-Dataset', one_hot=True)

In [ ]:
train_x_raw = mnist.train.images
train_y_raw = mnist.train.labels
valid_x_raw = mnist.validation.images
valid_y_raw = mnist.validation.labels
test_x_raw = mnist.test.images
test_y_raw = mnist.test.labels

## Explore Data

In [ ]:
def plot_mnist(data_x, data_y, n):
    fig = plt.figure(figsize=[16,9])
    for i in range(n):
        ax = fig.add_subplot(n//8, 8, i+1)
        ax.imshow(data_x[i].reshape([28,28]))
        ax.axis('off')
        idx = int(np.nonzero(data_y[i])[0])
        ax.set_title(idx)
    plt.show()

In [ ]:
plot_mnist(train_x_raw, train_y_raw, 32)

In [ ]:
plot_mnist(valid_x_raw, valid_y_raw, 8)

In [ ]:
plot_mnist(test_x_raw, test_y_raw, 8)

In [ ]:
plt.hist(train_x_raw[0:100].flatten(), bins=100);

In [ ]:
print('train mean:', train_x_raw.mean(), 'std', train_x_raw.std())
print('valid mean:', valid_x_raw.mean(), 'std', valid_x_raw.std())
print('test mean:', test_x_raw.mean(), 'std', test_x_raw.std())

## Preprocess Data

In [ ]:
tx_mean = train_x_raw.mean()
tx_std = train_x_raw.std()
train_x = (train_x_raw - tx_mean)/tx_std
valid_x = (valid_x_raw - tx_mean)/tx_std
test_x = (test_x_raw - tx_mean)/tx_std
train_y = train_y_raw
valid_y = valid_y_raw
test_y = test_y_raw
print('train mean:', train_x.mean(), 'std', train_x.std())
print('valid mean:', valid_x.mean(), 'std', valid_x.std())
print('test mean:', test_x.mean(), 'std', test_x.std())
print(train_x.shape, train_y.shape)
print(valid_x.shape, valid_y.shape)
print(test_x.shape, test_y.shape)

In [ ]:
plot_mnist(train_x, train_y, 8)

# Fast import/restart

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from tensorflow.examples.tutorials.mnist import input_data
import plotann
import pdb

np.random.seed(0)

mnist = input_data.read_data_sets('MNIST-Dataset', one_hot=True)

train_x_raw = mnist.train.images
train_y_raw = mnist.train.labels
valid_x_raw = mnist.validation.images
valid_y_raw = mnist.validation.labels
test_x_raw = mnist.test.images
test_y_raw = mnist.test.labels

# Preprocess
tx_mean = train_x_raw.mean()
tx_std = train_x_raw.std()
train_x = (train_x_raw - tx_mean)/tx_std
valid_x = (valid_x_raw - tx_mean)/tx_std
test_x = (test_x_raw - tx_mean)/tx_std
train_y = train_y_raw
valid_y = valid_y_raw
test_y = test_y_raw

In [ ]:
def subsample_to_14_14(data):
    assert data.ndim == 2
    assert data.shape[1] == 784
    data_28x28 = data.reshape([len(data), 28, 28])  # reshape to match image resolution, new shape (nb_samples, 28, 28)
    print('28', data_28x28.shape)
    data_14x14 = data_28x28[:,::2,::2] # subsample, new shape (nb_samples, 14, 14)
    print('14', data_14x14.shape)
    data_196 = data_14x14.reshape([len(data),14*14])
    print('data_196', data_196.shape)
    assert data_196.ndim == 2
    assert data_196.shape[1] == 196
    return data_196

In [ ]:
train_x = subsample_to_14_14(train_x)
valid_x = subsample_to_14_14(valid_x)
test_x = subsample_to_14_14(test_x)

# Activation Functions

These are used for numpy model, but also for plotting later on

In [ ]:
def sigmoid(x, deriv=False):
    if deriv:
        return sigmoid(x)*(1-sigmoid(x))
    return 1 / (1 + np.exp(-x))

def tanh(x, deriv=False):
    if deriv:
        return 1. - np.tanh(x)**2
    return np.tanh(x)

def softssign(x, deriv=False):
    if deriv:
        dd = 1 + np.abs(x)
        return (dd - x*np.sign(x)) / dd**2
    return x / (1+np.abs(x))

def relu(x, deriv=False):
    if deriv:
        return 1. * (x>0)
    return np.maximum(0, x)

def lrelu(x, deriv=False):
    if deriv:
        dx = np.ones_like(x)
        dx[x < 0] = 0.01
        return dx
    return np.where(x > 0, x, x * 0.01)

act_fun_dict = {'sigmoid': sigmoid, 'tanh': tanh, 'softssign':softssign,
                'relu':relu, 'lrelu':lrelu}

# Numpy Model

In [ ]:
def MSE(y, y_hat):
    assert y.ndim == 2
    assert y_hat.ndim == 2
    
    # avg over batch, sum over outputs (inner)
    return .5 * np.mean(np.sum((y-y_hat)**2, axis=-1))
    
    # no innner sum, becouse only one output
    return np.mean((y-y_hat)**2)

def acc(y, y_hat):
    return np.mean(np.argmax(y_hat, axis=-1)==np.argmax(y, axis=-1))

def fwd(x, W_hid, W_out, act_fun, ret=False):
    assert x.ndim == 2
    z_hid = x @ W_hid
    h_hid = act_fun(z_hid)  # hidden output

    z_out = h_hid @ W_out
    y_hat = sigmoid(z_out)  # SIGMOID!

    if ret:
        return y_hat, z_hid, h_hid, z_out
    return y_hat

def backprop(x, y, W_hid, W_out, act_fun):
    assert x.ndim == 2
    assert y.ndim == 2
    
    y_hat, z_hid, h_hid, z_out = fwd(x, W_hid, W_out, act_fun, ret=True)
    
    ro_out = (y-y_hat) * -1 * sigmoid(z_out, deriv=True)  # SIGMOID
    dW_out = h_hid.T @ ro_out / len(x)
    
    ro_hid = (ro_out @ W_out.T) * act_fun(z_hid, deriv=True)
    dW_hid = x.T @ ro_hid / len(x)
    
    return dW_hid, dW_out

### Numerical gradient check (optional)

In [ ]:
def ngrad(x, y, W_hid, W_out, act_fun):
    
    eps = 1e-6
    
    gW_hid = np.zeros_like(W_hid)
    for r in range(W_hid.shape[0]):
        for c in range(W_hid.shape[1]):
            W_hid_plus = W_hid.copy()
            W_hid_minus = W_hid.copy()
            W_hid_plus[r,c] += eps
            W_hid_minus[r,c] -= eps
            loss_plus = MSE(y, fwd(x, W_hid_plus, W_out, act_fun))
            loss_minus = MSE(y, fwd(x, W_hid_minus, W_out, act_fun))
            gW_hid[r,c] = (loss_plus-loss_minus) / (2*eps)

    gW_out = np.zeros_like(W_out)
    for r in range(W_out.shape[0]):
        for c in range(W_out.shape[1]):
            W_out_plus = W_out.copy()
            W_out_minus = W_out.copy()
            W_out_plus[r,c] += eps
            W_out_minus[r,c] -= eps
            loss_plus = MSE(y, fwd(x, W_hid, W_out_plus, act_fun))
            loss_minus = MSE(y, fwd(x, W_hid, W_out_minus, act_fun))
            gW_out[r,c] = (loss_plus-loss_minus) / (2*eps)
    return gW_hid, gW_out

In [ ]:
hid_dW, out_dW = backprop(train_x[0:3], train_y[0:3], W_hid, W_out, act_fun)
hid_ngW, out_ngW = ngrad(train_x[0:3], train_y[0:3], W_hid, W_out, act_fun)

In [ ]:
assert np.allclose(hid_dW, hid_ngW)
assert np.allclose(out_dW, out_ngW)

# Train Loop - with traces

In [ ]:
import tensorflow as tf

In [ ]:
n_in = 196   # 784
n_hid = 128  # 128  # 128             # sigmoid try 8, 128(def.), 2048
n_out = 10
lr = 0.03    # 0.03         # sigmoid try 10, 1(best), 0.03, 0.0003

np_or_tf = 'tf2'

n_batch = 100
act_fun = 'relu'
completed_epochs = 0

np.random.seed(0)

# Initialize weights
var_hid = np.sqrt(1/n_in)       # sigmoid try:  0.001,  sqrt(1/n_in),  1
var_out = np.sqrt(1/n_hid)
W_hid = np.random.normal(0.0, var_hid, [n_in, n_hid])
W_out = np.random.normal(0.0, var_out, [n_hid, n_out])

batches = {'iter':[], 'loss':[], 'acc':[]}
valids = {'iter':[], 'loss':[]}
traces = {'hid_z':[], 'out_z':[],
          'hid_dW':[], 'out_dW':[],
          'hid_W':[], 'out_W':[]}

In [ ]:
if np_or_tf == 'np':
    print('Skipping graph build')
    
elif np_or_tf == 'tf':
    print('Initializing TensorFlow graph')
    
    try:    sess.close()
    except: pass
    tf.reset_default_graph()
    
    nn_x = tf.placeholder(shape=[None, n_in], dtype=tf.float32)
    nn_y = tf.placeholder(shape=[None, n_out], dtype=tf.float32)

    nn_hid_W = tf.get_variable('nn_hid_W', shape=W_hid.shape, dtype=tf.float32, initializer=tf.constant_initializer(W_hid))
    nn_hid_z = tf.matmul(nn_x, nn_hid_W)
    if act_fun == 'sigmoid':
        nn_hid_h = tf.nn.sigmoid(nn_hid_z)
    elif act_fun == 'relu':
        nn_hid_h = tf.nn.relu(nn_hid_z)
    else:
        raise ValueError('unknown activation function')

    nn_out_W = tf.get_variable('nn_out_W', shape=W_out.shape, dtype=tf.float32, initializer=tf.constant_initializer(W_out))
    nn_out_z = tf.matmul(nn_hid_h, nn_out_W)
    nn_y_hat = tf.nn.sigmoid(nn_out_z)  # output always sigmoid

    nn_mse = .5 * tf.reduce_mean( tf.reduce_sum(tf.pow(nn_y-nn_y_hat, 2), axis=-1) )
    nn_acc = tf.reduce_mean( 
                tf.cast( 
                    tf.equal( tf.argmax(nn_y_hat, axis=-1), tf.argmax(nn_y, axis=-1) )
                , tf.float32)
            )

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
    # optimizer = tf.train.RMSPropOptimizer(learning_rate=lr)
    # train_op = optimizer.minimize(nn_mse)

    grads_and_vars = optimizer.compute_gradients(nn_mse)
    train_op = optimizer.apply_gradients(grads_and_vars)

    nn_hid_dW = grads_and_vars[0][0]
    nn_out_dW = grads_and_vars[1][0]

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
elif np_or_tf == 'tf2':
    print('Initializing TensorFlow graph - High Level API')
    
    try:    sess.close()
    except: pass
    tf.reset_default_graph()
    
    nn_x = tf.placeholder(shape=[None, n_in], dtype=tf.float32)
    nn_y = tf.placeholder(shape=[None, n_out], dtype=tf.float32)
    
    nn_hid_h = tf.layers.dense(inputs=nn_x,
                               units=n_hid,
                               activation=tf.nn.relu,
                               use_bias=False,
                               kernel_initializer=tf.constant_initializer(W_hid),
                               name='Hidden')
    nn_hid_z = tf.get_default_graph().get_tensor_by_name('Hidden/MatMul:0')
    
    nn_y_hat = tf.layers.dense(inputs=nn_hid_h,
                               units=n_out,
                               activation=tf.nn.sigmoid,
                               use_bias=False,
                               kernel_initializer=tf.constant_initializer(W_out),
                               name='Output')
    nn_out_z = tf.get_default_graph().get_tensor_by_name('Output/MatMul:0')
        
    nn_mse = .5 * tf.reduce_mean( tf.reduce_sum(tf.pow(nn_y-nn_y_hat, 2), axis=-1) )
    nn_acc = tf.reduce_mean( 
                tf.cast( 
                    tf.equal( tf.argmax(nn_y_hat, axis=-1), tf.argmax(nn_y, axis=-1) )
                , tf.float32)
            )
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
    
    grads_and_vars = optimizer.compute_gradients(nn_mse)
    train_op = optimizer.apply_gradients(grads_and_vars)
    
    nn_hid_dW = grads_and_vars[0][0]
    nn_out_dW = grads_and_vars[1][0]
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

In [ ]:
# writer = tf.summary.FileWriter(logdir='tf_log', graph=sess.graph)
# writer.flush()

In [ ]:
ti_ = 0
train_i = np.array(range(len(train_x)))
for e in range(10):
    print(e)
    np.random.shuffle(train_i)
            
    for i in range(0, len(train_x), n_batch):

        # Get 128 sized batch, both as 2d arrays   
        batch = train_i[i:i+n_batch]
        x = train_x[batch]
        y = train_y[batch]
        
        if np_or_tf == 'np':
            
            if ti_ == 0:
                print('Executing Numpy version')
            
            # Forward pass
            y_hat, hid_z, _, out_z = fwd(x, W_hid, W_out, act_fun_dict[act_fun], ret=True)
            loss = MSE(y, y_hat)
            accuracy = acc(y, y_hat)
            
            # Backpropagation
            hid_dW, out_dW = backprop(x, y, W_hid, W_out, act_fun_dict[act_fun])
            W_hid += -lr * hid_dW
            W_out += -lr * out_dW
            
        elif np_or_tf == 'tf':
            
            if ti_ == 0:
                print('Executing Tensorflow version')
            
            _, y_hat, hid_z, out_z, loss, accuracy, hid_dW, out_dW = sess.run(
                [train_op, nn_y_hat, nn_hid_z, nn_out_z, nn_mse, nn_acc, nn_hid_dW, nn_out_dW],
                feed_dict={nn_x: x, nn_y:y})
            W_hid, W_out = sess.run(tf.trainable_variables())
            
            loss_val = sess.run(nn_mse, feed_dict={nn_x: valid_x, nn_y:valid_y})
            
            assert y_hat.shape == y.shape
            assert np.isscalar(loss)
            assert np.isscalar(accuracy)
            
        elif np_or_tf == 'tf2':
            
            if ti_ == 0:
                print('Executing Tensorflow version - High Level API')
            
            _, y_hat, hid_z, out_z, loss, accuracy, hid_dW, out_dW = sess.run(
                [train_op, nn_y_hat, nn_hid_z, nn_out_z, nn_mse, nn_acc, nn_hid_dW, nn_out_dW],
                feed_dict={nn_x: x, nn_y:y})
            W_hid, W_out = sess.run(tf.trainable_variables())
            
            loss_val = sess.run(nn_mse, feed_dict={nn_x: valid_x, nn_y:valid_y})
            
            
            assert y_hat.shape == y.shape
            assert np.isscalar(loss)
            assert np.isscalar(accuracy)
            
            
        else:
            raise ValueError('np_or_tf must be "np" or "tf"')
        
        batches['iter'].append(ti_)
        batches['loss'].append(loss)
        batches['acc'].append(accuracy)
        
        valids['iter'].append(ti_)
        valids['loss'].append(loss_val)
        
        traces['hid_z'].append(hid_z)
        traces['out_z'].append(out_z)
        traces['hid_dW'].append(hid_dW)
        traces['out_dW'].append(out_dW)
        traces['hid_W'].append(W_hid.copy())
        traces['out_W'].append(W_out.copy())

        ti_ += 1
        
    completed_epochs += 1
    
expstr = 'Network(' + np_or_tf + '): '
expstr += str(n_in) + 'in->' + str(n_hid) + act_fun + '->' + str(n_out) + 'sig   '
expstr += 'init_var=[' + str(round(var_hid, 3)) + ',' + str(round(var_out,3))+ ']   '
expstr += 'lr=' + str(lr) + '   '
expstr += 'batch=' + str(n_batch) + '   '
expstr += 'epochs=' + str(completed_epochs)

tr_hid_z = np.array(traces['hid_z'])
tr_out_z = np.array(traces['out_z'])
tr_hid_dW = np.array(traces['hid_dW'])
tr_out_dW = np.array(traces['out_dW'])
tr_hid_W = np.array(traces['hid_W'])
tr_out_W = np.array(traces['out_W'])

print('tr_hid_z', tr_hid_z.shape, tr_hid_z.size/1e6)
print('tr_out_z', tr_out_z.shape, tr_out_z.size/1e6)
print('tr_hid_dW', tr_hid_dW.shape, tr_hid_dW.size/1e6)
print('tr_out_dW', tr_out_dW.shape, tr_out_dW.size/1e6)
print('tr_hid_W', tr_hid_W.shape, tr_hid_W.size/1e6)
print('tr_out_W', tr_out_W.shape, tr_out_W.size/1e6)

print(expstr)

# Loss, output

In [ ]:
#
#   Plot Loss, Accuracy
#
print(expstr)
fig, ax = plt.subplots(figsize=[12,6])
ax.plot(batches['iter'], batches['loss'], label='Mini-Batch loss', alpha=.5)
ax.plot(batches['iter'], batches['acc'], label='Mini-Batch accuracy', color='red', alpha=.5)
ax.plot(valids['iter'], valids['loss'], label='Validation loss', alpha=.5)

ax.legend()
ax.set_ylim(0, 1)
ax.grid()

plt.tight_layout()
plt.show;

---

In [ ]:
def plot_class_coverage(y, y_hat, axis=None):
    #
    #   Per-class accuracy
    #
    assert y.ndim == 2 and y_hat.ndim == 2
    assert y.shape == y_hat.shape
    
    nb_classses = y.shape[-1]
    
    y_correct = np.argmax(y, axis=-1) == np.argmax(y_hat, axis=-1)
    classes_correct = []
    classes_all = []
    for i in range(nb_classses):
        is_y_class_i = y_correct * (np.argmax(y, axis=-1)==i)
        nb_correct_class_i = np.sum(is_y_class_i)
        classes_correct.append(nb_correct_class_i)
        classes_all.append(np.count_nonzero(np.argmax(y, axis=-1)==i))
    classes_correct = np.array(classes_correct)
    classes_all = np.array(classes_all)

    if axis is None:
        fig, axis = plt.subplots()
    
    axis.bar(range(10), classes_all, label='All Member')
    axis.bar(range(10), classes_correct, label='Correctly Predicted')
    axis.legend(loc=3)

In [ ]:
data_x, data_y = train_x, train_y
if np_or_tf == 'np':              y_hat = fwd(data_x, W_hid, W_out, act_fun_dict[act_fun])
elif np_or_tf in ['tf', 'tf2']:   y_hat = sess.run(nn_y_hat, feed_dict={nn_x: data_x})
else:                             raise ValueError()
plot_class_coverage(data_y, y_hat)

In [ ]:
data_x, data_y = valid_x, valid_y
if np_or_tf == 'np':              y_hat = fwd(data_x, W_hid, W_out, act_fun_dict[act_fun])
elif np_or_tf in ['tf', 'tf2']:   y_hat = sess.run(nn_y_hat, feed_dict={nn_x: data_x})
else:                             raise ValueError()
plot_class_coverage(data_y, y_hat)

# Weight Plots

In [ ]:
def running_mean(x, n):
    return np.array([ np.mean(x[max(i-n+1, 0): i+1]) for i in range(len(x))])

In [ ]:
def running_std(x, n):
    return np.array([ np.std(x[max(i-n+1, 0): i+1]) for i in range(len(x))])

---

### Hidden

In [ ]:
plotann.show_neurons_weights(tr_hid_W, tr_hid_dW, neurons=range(3),
                            title_prefix='Hidden', color='red', figsize=[16,4])

In [ ]:
plotann.show_layer_summary(tr_hid_W, tr_hid_dW,
                           title_prefix='Hidden', color='red', figsize=[16,5])

### Output

In [ ]:
plotann.show_neurons_weights(tr_out_W, tr_out_dW, neurons=range(tr_out_W.shape[-1]),
                            title_prefix='Output', color='blue', figsize=[16,4])

In [ ]:
plotann.show_layer_summary(tr_out_W, tr_out_dW, title_prefix='Output', color='blue', figsize=[16,5])

# Activations

In [ ]:
es=55000

### Hidden

In [ ]:
plotann.show_layer_activations(tr_hid_z, epoch_size=es, activation_function=act_fun,
                               title_prefix='Hidden', color=(1,0,0,1), figsize=[16,6])

In [ ]:
plotann.show_neurons_activations(tr_hid_z, epoch_size=es, activation_function=act_fun, neurons=range(10),
                                 title_prefix='Hidden', color=(1,0,0,1))

---

### Output

In [ ]:
plotann.show_layer_activations(tr_out_z, epoch_size=es, activation_function='sigmoid',
                               title_prefix='Output', color=(0,0,1,1), figsize=[16,6])

In [ ]:
plotann.show_neurons_activations(tr_out_z, epoch_size=es, activation_function='sigmoid', neurons=range(tr_out_z.shape[-1]),
                                 title_prefix='Output', color=(0,0,1,1))